# Fuzzysearch experiments

In [3]:
# not for my case, it just gives score
#import jellyfish

# not for my case, it gives substring
#import fuzzysearch

# i did not get how it differs from it's father lib -- rapidfuzz
#import thefuzz

# from `import pynndescent`
#   AttributeError: `np.infty` was removed in the NumPy 2.0 release. Use `np.inf` instead.
#import neofuzz

# import rapidfuzz 
from rapidfuzz import process

In [4]:
import rapidfuzz

In [7]:
import json

In [8]:
# from 002-kba notebook
with open('../fuzzy_sidc/set_b_2525d.json', 'r') as fp:
    data =  json.load(fp)

In [10]:
choices = [i.lower() for i in data.keys()]

In [12]:
# choices

## different scorers

In [13]:
# default
# all other scorers, depending on len1/len2 ratio
# good thing, but not for replicative things
process.extract("mortar.armored", choices, limit=2, scorer=rapidfuzz.fuzz.WRatio)

[('land unit.fires.mortar.armored/mechanized/tracked', 90.0, 117),
 ('land unit.movement and maneuver.antitank/antiarmor.armored', 79.2, 68)]

In [14]:
# Compares strings
# Calculates the normalized Indel similarity (like Levenshtein distance, but does not allow substitutions)
# Since v3.0 this behaves similar to fuzz.ratio 
#  with the exception that this returns 0 when comparing two empty strings
# speeds are the same
process.extract("mortar.armored", choices, limit=2, scorer=rapidfuzz.fuzz.QRatio)

[('land unit.modifier_1.armored', 52.38095238095239, 380),
 ('land unit.fires.mortar.towed', 47.61904761904761, 119)]

In [15]:
# same as QRatio
process.extract("mortar.armored", choices, limit=2, scorer=rapidfuzz.fuzz.ratio)

[('land unit.modifier_1.armored', 52.38095238095239, 380),
 ('land unit.fires.mortar.towed', 47.61904761904761, 119)]

In [16]:
# Compares the words in the strings
# no order at all -- set intersection
# not for two unsplitable strings
process.extract("mortar.armored", choices, limit=2, scorer=rapidfuzz.fuzz.token_set_ratio)

[('land unit.modifier_1.armored', 52.38095238095238, 380),
 ('land unit.fires.mortar.towed', 47.61904761904762, 119)]

In [17]:
# Compares the words in the strings
# use fuzz.ratio on lists of sorted words (aka tokens)
# the most bizzare thing
process.extract("mortar.armored", choices, limit=2, scorer=rapidfuzz.fuzz.token_sort_ratio)

[('land unit.modifier_1.armored', 52.38095238095239, 380),
 ('land unit.fires.mortar.towed', 47.61904761904761, 119)]

In [18]:
# Compares strings
# WHAT WE NEED
# search for optimal substring
process.extract("mortar.armored", choices, limit=2, scorer=rapidfuzz.fuzz.partial_ratio)

[('land unit.fires.mortar.armored/mechanized/tracked', 100.0, 117),
 ('land unit.movement and maneuver.antitank/antiarmor.armored', 88.0, 68)]

In [19]:
# partial_ratio_alignment just provides info, cannot be used as a scorer
# process.extract("mortar.armored", choices, limit=2, scorer=rapidfuzz.fuzz.partial_ratio_alignment)

In [20]:
# Compares the words in the strings
# Similar to token_set_ratio(), but it uses partial token matching
process.extract("mortar.armored", choices, limit=2, scorer=rapidfuzz.fuzz.partial_token_set_ratio)

[('land unit.fires.mortar.armored/mechanized/tracked', 100.0, 117),
 ('land unit.movement and maneuver.antitank/antiarmor.armored',
  78.57142857142857,
  68)]

In [21]:
# Compares the words in the strings
# A combination of partial_ratio() and token_sort_ratio(),
#   for sorting and then partially matching.
process.extract("mortar.armored", choices, limit=2, scorer=rapidfuzz.fuzz.partial_token_sort_ratio)

[('land unit.fires.mortar.armored/mechanized/tracked', 100.0, 117),
 ('land unit.movement and maneuver.antitank/antiarmor.armored',
  78.57142857142857,
  68)]

## make choices tokenable

In [22]:
# make choices tokenable
choices = [i.lower().replace('.', ' ') for i in data.keys()]

In [23]:
# Compares strings
# WHAT WE NEED
# search for optimal substring
process.extract("mortar armored", choices, limit=2, scorer=rapidfuzz.fuzz.partial_ratio)

[('land unit fires mortar armored/mechanized/tracked', 100.0, 117),
 ('land unit movement and maneuver antitank/antiarmor armored', 88.0, 68)]

In [24]:
# Compares the words in the strings
# A combination of partial_ratio() and token_sort_ratio(),
#   for sorting and then partially matching.
process.extract("mortar armored", choices, limit=2, scorer=rapidfuzz.fuzz.partial_token_sort_ratio)

[('land equipment vehicle armored armored fighting vehicle',
  83.33333333333334,
  2072),
 ('land equipment vehicle armored armored personnel carrier',
  83.33333333333334,
  2074)]

In [25]:
# Compares the words in the strings
# A combination of partial_ratio() and token_sort_ratio(),
#   for sorting and then partially matching.
process.extract("armored mortar", choices, limit=2, scorer=rapidfuzz.fuzz.partial_token_sort_ratio)

[('land equipment vehicle armored armored fighting vehicle',
  83.33333333333334,
  2072),
 ('land equipment vehicle armored armored personnel carrier',
  83.33333333333334,
  2074)]

In [26]:
# Compares the words in the strings
# A combination of partial_ratio() and token_sort_ratio(),
#   for sorting and then partially matching.
process.extract("mortar armored", choices, limit=2, scorer=rapidfuzz.fuzz.partial_token_set_ratio)

[('land unit movement and maneuver antitank/antiarmor armored', 100.0, 68),
 ('land unit fires mortar ', 100.0, 116)]

## on same score, always chose 1st, Undefined always 1st

In [28]:
process.extractOne("mortar", choices)

('land unit fires mortar ', 90.0, 116)

In [29]:
process.extract("mortar", choices)

[('land unit fires mortar ', 90.0, 116),
 ('land unit fires mortar self-propelled wheeled', 90.0, 118),
 ('land unit fires mortar towed', 90.0, 119),
 ('land equipment weapon/weapon system mortar ', 90.0, 2051),
 ('land unit sustainment mortuary affairs', 75.00000000000001, 191)]

In [30]:
process.extract("mortar", choices, limit=2, scorer=rapidfuzz.fuzz.partial_ratio)

[('land unit fires mortar ', 100.0, 116),
 ('land unit fires mortar armored/mechanized/tracked', 100.0, 117)]

In [31]:
process.extract("mortar", choices, limit=2, scorer=rapidfuzz.fuzz.partial_token_set_ratio)

[('land unit fires mortar ', 100.0, 116),
 ('land unit fires mortar armored/mechanized/tracked', 100.0, 117)]

In [32]:
process.extract("mortar armored", choices, limit=2, scorer=rapidfuzz.fuzz.partial_token_sort_ratio)

[('land equipment vehicle armored armored fighting vehicle',
  83.33333333333334,
  2072),
 ('land equipment vehicle armored armored personnel carrier',
  83.33333333333334,
  2074)]

## SIDC set_B obtaining

In [33]:
choices = data.keys()

In [34]:
data[process.extractOne("mortar", choices)[0]]

'130800'